In [1]:
import sklearn.datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
readdata=pd.read_csv(r"C:\Users\adhit\ML Data sets\Inhouse project\heartv1.csv")
data = pd.DataFrame(readdata)
#data = data.drop([87,166,192,266,287,302],axis=0)
print(data.head())
print(data.shape)
print(data['target'].value_counts())

   age  cp  thalach  oldpeak  ca  thal  target
0   52   0      168      1.0   2     3       0
1   53   0      155      3.1   0     3       0
2   70   0      125      2.6   0     3       0
3   61   0      161      0.0   1     3       0
4   62   0      106      1.9   3     2       0
(1025, 7)
1    526
0    499
Name: target, dtype: int64


In [3]:
data.describe()

,age,cp,thalach,oldpeak,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.942439,149.114146,1.071512,0.754146,2.323902,0.513171
std,9.072290,1.029641,23.005724,1.175053,1.030798,0.620660,0.500070
min,29.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,132.000000,0.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,152.000000,0.800000,0.000000,2.000000,1.000000
75%,61.000000,2.000000,166.000000,1.800000,1.000000,3.000000,1.000000
max,77.000000,3.000000,202.000000,6.200000,4.000000,3.000000,1.000000


In [6]:
from sklearn.model_selection import train_test_split
newdata=data.dropna(axis=0,how='any')
print(newdata.shape)
X = data.drop('target', axis=1)
Y = data['target']
Y1=list(Y)
#Y = list(map(lambda x: 1 if x>0 else 0,Y1))
#newx=pd.DataFrame(data.ix[:,[0,1,2,4,5,8,9,10,11]])
#newx=data[data.columns[6]]
#print(newx)
#X = newx
Y=pd.DataFrame(Y)
print(X.shape)
print(Y.shape)
print(Y)
type(X)
type(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify = Y, random_state=1)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)
print(X_train.mean(), X_test.mean(), X.mean())

(1025, 7)
(1025, 6)
(1025, 1)
      target
0          0
1          0
2          0
3          0
4          0
...      ...
1020       1
1021       0
1022       0
1023       1
1024       0

[1025 rows x 1 columns]
(717, 6) (308, 6) (717, 1) (308, 1)
age         54.334728
cp           0.921897
thalach    149.377964
oldpeak      1.071967
ca           0.754533
thal         2.327755
dtype: float64 age         54.665584
cp           0.990260
thalach    148.500000
oldpeak      1.070455
ca           0.753247
thal         2.314935
dtype: float64 age         54.434146
cp           0.942439
thalach    149.114146
oldpeak      1.071512
ca           0.754146
thal         2.323902
dtype: float64


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
lcclassifier = LogisticRegression()  
lcclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = lcclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print(accuracy*100)
print(precision*100)
print(recall*100)
print(f1)
print("Bagging Classifier")
baggclassifier=BaggingClassifier(base_estimator=lcclassifier,n_estimators=300)
print("Boosting Classifier")
boostclassifier=AdaBoostClassifier(base_estimator=lcclassifier,algorithm="SAMME",n_estimators=300)
def run_kfold(clf):
    kf = KFold(10,shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, np.ravel(y_train))
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))
        
run_kfold(boostclassifier)

81.81818181818183
89.24050632911393
78.33333333333333
0.8343195266272189
Bagging Classifier
Boosting Classifier
Fold 1 accuracy: 0.8446601941747572
Fold 1 precision: 0.84
Fold 1 recall: 0.84
Fold 1 f1: 0.8399999999999999
Mean Accuracy: 0.8446601941747572
Mean Precision: 0.84
Mean Recall: 0.84
Mean F1: 0.8399999999999999
Fold 2 accuracy: 0.9223300970873787
Fold 2 precision: 0.8983050847457628
Fold 2 recall: 0.9636363636363636
Fold 2 f1: 0.9298245614035089
Mean Accuracy: 0.883495145631068
Mean Precision: 0.8691525423728814
Mean Recall: 0.9018181818181819
Mean F1: 0.8849122807017544
Fold 3 accuracy: 0.8932038834951457
Fold 3 precision: 0.9016393442622951
Fold 3 recall: 0.9166666666666666
Fold 3 f1: 0.9090909090909091
Mean Accuracy: 0.8867313915857605
Mean Precision: 0.8799814763360193
Mean Recall: 0.9067676767676768
Mean F1: 0.8929718234981393
Fold 4 accuracy: 0.8349514563106796
Fold 4 precision: 0.8461538461538461
Fold 4 recall: 0.8301886792452831
Fold 4 f1: 0.8380952380952382
Mean Accur

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
knnclassifier = KNeighborsClassifier(n_neighbors=2)  
knnclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = knnclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print(accuracy*100)
print(precision*100)
print(recall*100)
print(f1)
print("Bagging Classifier")
baggclassifier=BaggingClassifier(base_estimator=knnclassifier,n_estimators=300)
print("Boosting Classifier")
boostclassifier=AdaBoostClassifier(base_estimator=knnclassifier,algorithm="SAMME",n_estimators=150)
print("K Fold Validation")
def run_kfold(clf):
    kf = KFold(10,shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, np.ravel(y_train))
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))
run_kfold(baggclassifier)

91.23376623376623
84.81012658227847
97.8102189781022
0.9084745762711864
Bagging Classifier
Boosting Classifier
K Fold Validation
Fold 1 accuracy: 1.0
Fold 1 precision: 1.0
Fold 1 recall: 1.0
Fold 1 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 2 accuracy: 0.9805825242718447
Fold 2 precision: 0.9649122807017544
Fold 2 recall: 1.0
Fold 2 f1: 0.9821428571428572
Mean Accuracy: 0.9902912621359223
Mean Precision: 0.9824561403508771
Mean Recall: 1.0
Mean F1: 0.9910714285714286
Fold 3 accuracy: 0.9805825242718447
Fold 3 precision: 1.0
Fold 3 recall: 0.9666666666666667
Fold 3 f1: 0.983050847457627
Mean Accuracy: 0.9870550161812298
Mean Precision: 0.9883040935672515
Mean Recall: 0.9888888888888889
Mean F1: 0.9883979015334948
Fold 4 accuracy: 0.970873786407767
Fold 4 precision: 1.0
Fold 4 recall: 0.9433962264150944
Fold 4 f1: 0.970873786407767
Mean Accuracy: 0.9830097087378641
Mean Precision: 0.9912280701754386
Mean Recall: 0.9775157232704403
Mean F1: 0.9840168

In [7]:
from sklearn.svm import SVC  
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
svclassifier = SVC(kernel='linear')  
svclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = svclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print("Accuracy=",accuracy*100)
print("Precision=",precision*100)
print("Recall=",recall*100)
print("F1=",f1)
print("Bagging Classifier")
baggclassifier=BaggingClassifier(base_estimator=svclassifier,n_estimators=300)
print("Boosting Classifier")
boostclassifier=AdaBoostClassifier(base_estimator=svclassifier,algorithm="SAMME",n_estimators=150)
print("K Fold Validation")
def run_kfold(clf):
    #kf = KFold(303, n_folds=10)
    kf = KFold(10,shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, np.ravel(y_train))
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))

run_kfold(boostclassifier)

Accuracy= 81.16883116883116
Precision= 89.24050632911393
Recall= 77.47252747252747
F1= 0.8294117647058824
Bagging Classifier
Boosting Classifier
K Fold Validation
Fold 1 accuracy: 0.7961165048543689
Fold 1 precision: 0.7457627118644068
Fold 1 recall: 0.88
Fold 1 f1: 0.8073394495412843
Mean Accuracy: 0.7961165048543689
Mean Precision: 0.7457627118644068
Mean Recall: 0.88
Mean F1: 0.8073394495412843
Fold 2 accuracy: 0.9223300970873787
Fold 2 precision: 0.8852459016393442
Fold 2 recall: 0.9818181818181818
Fold 2 f1: 0.9310344827586207
Mean Accuracy: 0.8592233009708738
Mean Precision: 0.8155043067518755
Mean Recall: 0.9309090909090909
Mean F1: 0.8691869661499525
Fold 3 accuracy: 0.8737864077669902
Fold 3 precision: 0.8852459016393442
Fold 3 recall: 0.9
Fold 3 f1: 0.8925619834710743
Mean Accuracy: 0.8640776699029127
Mean Precision: 0.8387515050476985
Mean Recall: 0.9206060606060605
Mean F1: 0.8769786385903263
Fold 4 accuracy: 0.7281553398058253
Fold 4 precision: 0.711864406779661
Fold 4 rec

In [8]:
from sklearn.svm import SVC  
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
svclassifier = SVC(kernel='rbf')  
svclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = svclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print("Accuracy=",accuracy*100)
print("Precision=",precision*100)
print("Recall=",recall*100)
print("F1=",f1)
print("Bagging Classifier")
baggclassifier=BaggingClassifier(base_estimator=svclassifier,n_estimators=300)
print("Boosting Classifier")
boostclassifier=AdaBoostClassifier(base_estimator=svclassifier,algorithm="SAMME",n_estimators=150)
print("K Fold Validation")
def run_kfold(clf):
    #kf = KFold(303, n_folds=10)
    kf = KFold(10,shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, np.ravel(y_train))
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))

run_kfold(boostclassifier)


Accuracy= 72.07792207792207
Precision= 74.68354430379746
Recall= 71.95121951219512
F1= 0.732919254658385
Bagging Classifier
Boosting Classifier
K Fold Validation
Fold 1 accuracy: 0.4854368932038835
Fold 1 precision: 0.4854368932038835
Fold 1 recall: 1.0
Fold 1 f1: 0.6535947712418301
Mean Accuracy: 0.4854368932038835
Mean Precision: 0.4854368932038835
Mean Recall: 1.0
Mean F1: 0.6535947712418301
Fold 2 accuracy: 0.5339805825242718
Fold 2 precision: 0.5339805825242718
Fold 2 recall: 1.0
Fold 2 f1: 0.6962025316455696
Mean Accuracy: 0.5097087378640777
Mean Precision: 0.5097087378640777
Mean Recall: 1.0
Mean F1: 0.6748986514436999
Fold 3 accuracy: 0.5825242718446602
Fold 3 precision: 0.5825242718446602
Fold 3 recall: 1.0
Fold 3 f1: 0.7361963190184049
Mean Accuracy: 0.5339805825242718
Mean Precision: 0.5339805825242718
Mean Recall: 1.0
Mean F1: 0.6953312073019348
Fold 4 accuracy: 0.5145631067961165
Fold 4 precision: 0.5145631067961165
Fold 4 recall: 1.0
Fold 4 f1: 0.6794871794871795
Mean Acc

In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
#from sklearn.cross_validation import KFold
nbclassifier = GaussianNB()  
nbclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = nbclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print(accuracy*100)
print(precision*100)
print(recall*100)
print(f1)

print("Bagging Classifier")
baggclassifier=BaggingClassifier(base_estimator=nbclassifier,n_estimators=300)
print("Boosting Classifier")
boostclassifier=AdaBoostClassifier(base_estimator=nbclassifier,algorithm="SAMME",n_estimators=600)
def run_kfold(clf):
    kf = KFold(10,shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, np.ravel(y_train))
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))
        
run_kfold(boostclassifier)

85.06493506493507
89.24050632911393
82.94117647058825
0.8597560975609756
Bagging Classifier
Boosting Classifier
Fold 1 accuracy: 0.8640776699029126
Fold 1 precision: 0.86
Fold 1 recall: 0.86
Fold 1 f1: 0.8599999999999999
Mean Accuracy: 0.8640776699029126
Mean Precision: 0.86
Mean Recall: 0.86
Mean F1: 0.8599999999999999
Fold 2 accuracy: 0.883495145631068
Fold 2 precision: 0.8524590163934426
Fold 2 recall: 0.9454545454545454
Fold 2 f1: 0.8965517241379309
Mean Accuracy: 0.8737864077669903
Mean Precision: 0.8562295081967213
Mean Recall: 0.9027272727272727
Mean F1: 0.8782758620689655
Fold 3 accuracy: 0.8932038834951457
Fold 3 precision: 0.8769230769230769
Fold 3 recall: 0.95
Fold 3 f1: 0.912
Mean Accuracy: 0.8802588996763755
Mean Precision: 0.8631273644388399
Mean Recall: 0.9184848484848485
Mean F1: 0.8895172413793103
Fold 4 accuracy: 0.7766990291262136
Fold 4 precision: 0.7678571428571429
Fold 4 recall: 0.8113207547169812
Fold 4 f1: 0.7889908256880735
Mean Accuracy: 0.854368932038835
Mean

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
nnclassifier = MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(13,15,20,15,13), random_state=1, max_iter=200)
nnclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = nnclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print(accuracy*100)
print(precision*100)
print(recall*100)
print(f1)

print("Bagging Classifier")
baggclassifier=BaggingClassifier(base_estimator=nnclassifier,n_estimators=300)
print("Boosting Classifier")
boostclassifier=AdaBoostClassifier(base_estimator=nnclassifier,algorithm="SAMME",n_estimators=150)

def run_kfold(clf):
    kf = KFold(10, shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, np.ravel(y_train))
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))
        
run_kfold(baggclassifier)

83.76623376623377
89.87341772151899
80.68181818181817
0.8502994011976047
Bagging Classifier
Boosting Classifier


KeyboardInterrupt: 

In [13]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
#from sklearn.cross_validation import KFold
xgclassifier = XGBClassifier(eval_metric='mlogloss')
xgclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = xgclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print(accuracy*100)
print(precision*100)
print(recall*100)
print(f1)

def run_kfold(clf):
    kf = KFold(10,shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))
        
run_kfold(xgclassifier)

99.02597402597402
98.10126582278481
100.0
0.9904153354632589
Fold 1 accuracy: 1.0
Fold 1 precision: 1.0
Fold 1 recall: 1.0
Fold 1 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 2 accuracy: 1.0
Fold 2 precision: 1.0
Fold 2 recall: 1.0
Fold 2 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 3 accuracy: 1.0
Fold 3 precision: 1.0
Fold 3 recall: 1.0
Fold 3 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 4 accuracy: 1.0
Fold 4 precision: 1.0
Fold 4 recall: 1.0
Fold 4 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 5 accuracy: 1.0
Fold 5 precision: 1.0
Fold 5 recall: 1.0
Fold 5 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 6 accuracy: 1.0
Fold 6 precision: 1.0
Fold 6 recall: 1.0
Fold 6 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 7 accuracy: 0.9705882352941176
Fold 7 precision: 1.0
Fold 

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
dtclassifier = DecisionTreeClassifier()  
dtclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = dtclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print(accuracy*100)
print(precision*100)
print(recall*100)
print(f1)
print(sensitivity)
print(specificity)
print("Bagging Classifier")
baggclassifier=BaggingClassifier(base_estimator=dtclassifier,n_estimators=300)
print("Boosting Classifier")
boostclassifier=AdaBoostClassifier(base_estimator=dtclassifier,algorithm="SAMME",n_estimators=150)
def run_kfold(clf):
    kf = KFold(10,shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, np.ravel(y_train))
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))
        
run_kfold(boostclassifier)

100.0
100.0
100.0
1.0
1.0
0.0
Bagging Classifier
Boosting Classifier
Fold 1 accuracy: 1.0
Fold 1 precision: 1.0
Fold 1 recall: 1.0
Fold 1 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 2 accuracy: 1.0
Fold 2 precision: 1.0
Fold 2 recall: 1.0
Fold 2 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 3 accuracy: 1.0
Fold 3 precision: 1.0
Fold 3 recall: 1.0
Fold 3 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 4 accuracy: 1.0
Fold 4 precision: 1.0
Fold 4 recall: 1.0
Fold 4 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 5 accuracy: 1.0
Fold 5 precision: 1.0
Fold 5 recall: 1.0
Fold 5 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 6 accuracy: 1.0
Fold 6 precision: 1.0
Fold 6 recall: 1.0
Fold 6 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 7 accuracy: 1.0
Fold 7 precision: 1.0
Fold 7 recal

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
rfclassifier = RandomForestClassifier(n_estimators=100)  
rfclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = rfclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print(accuracy*100)
print(precision*100)
print(recall*100)
print(f1)
print(sensitivity)
print(specificity)
print("Bagging Classifier")
baggclassifier=BaggingClassifier(base_estimator=rfclassifier,n_estimators=300)
print("Boosting Classifier")
boostclassifier=AdaBoostClassifier(base_estimator=rfclassifier,algorithm="SAMME",n_estimators=150)
def run_kfold(clf):
    kf = KFold(10,shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, np.ravel(y_train))
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))
        
run_kfold(boostclassifier)

100.0
100.0
100.0
1.0
1.0
0.0
Bagging Classifier
Boosting Classifier
Fold 1 accuracy: 1.0
Fold 1 precision: 1.0
Fold 1 recall: 1.0
Fold 1 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 2 accuracy: 1.0
Fold 2 precision: 1.0
Fold 2 recall: 1.0
Fold 2 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 3 accuracy: 1.0
Fold 3 precision: 1.0
Fold 3 recall: 1.0
Fold 3 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 4 accuracy: 1.0
Fold 4 precision: 1.0
Fold 4 recall: 1.0
Fold 4 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 5 accuracy: 1.0
Fold 5 precision: 1.0
Fold 5 recall: 1.0
Fold 5 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 6 accuracy: 1.0
Fold 6 precision: 1.0
Fold 6 recall: 1.0
Fold 6 f1: 1.0
Mean Accuracy: 1.0
Mean Precision: 1.0
Mean Recall: 1.0
Mean F1: 1.0
Fold 7 accuracy: 1.0
Fold 7 precision: 1.0
Fold 7 recal

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
gbclassifier = GradientBoostingClassifier()  
gbclassifier.fit(X_train, np.ravel(Y_train)) 
y_pred = gbclassifier.predict(X_test) 
accuracy=accuracy_score(y_pred,Y_test)
precision=precision_score(y_pred,Y_test)
recall=recall_score(y_pred,Y_test)
f1=2*(precision*recall)/(precision+recall)
cm=confusion_matrix(Y_test,y_pred)
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
specificity=cm[1,0]/(cm[1,0]+cm[1,1])
print(accuracy*100)
print(precision*100)
print(recall*100)
print(f1)
print(sensitivity)
print(specificity)
print("Bagging Classifier")
baggclassifier=BaggingClassifier(base_estimator=gbclassifier,n_estimators=300)
print("Boosting Classifier")
boostclassifier=AdaBoostClassifier(base_estimator=gbclassifier,algorithm="SAMME",n_estimators=150)
def run_kfold(clf):
    kf = KFold(10,shuffle=False)
    outcomesaccuracy = []
    outcomesprecision = []
    outcomesrecall = []
    outcomesf1 = []
    fold = 0
    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = Y.values[train_index], Y.values[test_index]
        clf.fit(X_train, np.ravel(y_train))
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions)
        recall=recall_score(y_test,predictions)
        f1=2*(precision*recall)/(precision+recall)
        cm=confusion_matrix(Y_test,y_pred)
        sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
        specificity=cm[1,0]/(cm[1,0]+cm[1,1])
        outcomesaccuracy.append(accuracy)
        outcomesprecision.append(precision)
        outcomesrecall.append(recall)
        outcomesf1.append(f1)
        print("Fold {0} accuracy: {1}".format(fold, accuracy)) 
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        mean_accuracy_outcome = np.mean(outcomesaccuracy)
        mean_precision_outcome = np.mean(outcomesprecision)
        mean_recall_outcome = np.mean(outcomesrecall)
        mean_f1_outcome = np.mean(outcomesf1)
        print("Mean Accuracy: {0}".format(mean_accuracy_outcome)) 
        print("Mean Precision: {0}".format(mean_precision_outcome))
        print("Mean Recall: {0}".format(mean_recall_outcome))
        print("Mean F1: {0}".format(mean_f1_outcome))
        
run_kfold(baggclassifier)

94.15584415584416
94.30379746835443
94.30379746835443
0.9430379746835443
0.94
0.056962025316455694
Bagging Classifier
Boosting Classifier
Fold 1 accuracy: 0.970873786407767
Fold 1 precision: 0.9607843137254902
Fold 1 recall: 0.98
Fold 1 f1: 0.9702970297029702
Mean Accuracy: 0.970873786407767
Mean Precision: 0.9607843137254902
Mean Recall: 0.98
Mean F1: 0.9702970297029702
Fold 2 accuracy: 0.970873786407767
Fold 2 precision: 0.9814814814814815
Fold 2 recall: 0.9636363636363636
Fold 2 f1: 0.9724770642201834
Mean Accuracy: 0.970873786407767
Mean Precision: 0.9711328976034859
Mean Recall: 0.9718181818181818
Mean F1: 0.9713870469615768
Fold 3 accuracy: 0.9805825242718447
Fold 3 precision: 0.9833333333333333
Fold 3 recall: 0.9833333333333333
Fold 3 f1: 0.9833333333333333
Mean Accuracy: 0.9741100323624595
Mean Precision: 0.975199709513435
Mean Recall: 0.9756565656565656
Mean F1: 0.975369142418829
Fold 4 accuracy: 0.9320388349514563
Fold 4 precision: 0.9791666666666666
Fold 4 recall: 0.88679245